In [ ]:
# Import same packages as the train script in Wave-U-Net-Pytorch
import argparse
import os
import time
from functools import partial

import torch
import pickle
import numpy as np

import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.optim import Adam
from torch.nn import L1Loss
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
# install torchaudio if not already installed
# ! pip install torchaudio
import torchaudio

import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional

!pip install sktime
from sktime.transformations.panel.rocket import MiniRocketMultivariate

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/git_projects/spring_2025_dl_audio_project

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 677.8 kB/s eta 0:00:00


In [ ]:
class MusdbDataset(Dataset):

  def __init__(self, musDB, steps = 256):
    self.mel_specs = torch.zeros(1, 2, 128, steps)
    self.sample_rates = torch.tensor([0])

    print("Tracks in MusDB:", len(musDB))

    for track in musDB:
      stems, rate = track.stems, track.rate

      # separate the vocal from other instruments and conver to mono signal
      audio_novocal = librosa.to_mono(np.transpose(stems[1] + stems[2] + stems[3]))
      audio_vocal = librosa.to_mono(np.transpose(stems[4]))

      # compute log mel spectrogram and convert to pytorch tensor
      logmelspec_novocal = torch.from_numpy(self._mel_spectrogram(audio_novocal, rate))
      logmelspec_vocal = torch.from_numpy(self._mel_spectrogram(audio_vocal, rate))

      num_slices = logmelspec_novocal.shape[1] // steps

      # chop off the last bit so that number of stft steps is a multiple of step size
      logmelspec_novocal = logmelspec_novocal[0:128 , 0:num_slices*steps]
      logmelspec_vocal = logmelspec_vocal[0:128, 0:num_slices*steps]

      logmelspec_novocal = torch.reshape(logmelspec_novocal, (num_slices, 128, steps))
      logmelspec_vocal = torch.reshape(logmelspec_vocal, (num_slices, 128, steps))

      # unsqueeze and concatenate these tensors. Then concatenate to the big tensor
      logmels = torch.cat((logmelspec_novocal.unsqueeze(1), logmelspec_vocal.unsqueeze(1)), 1)
      self.mel_specs = torch.cat((self.mel_specs, logmels), 0)
      self.sample_rates = torch.cat((self.sample_rates, torch.Tensor([rate])), 0)

    # remove the all zeros slice that we initialized with
    self.mel_specs = self.mel_specs[1: , : , : , :]
    self.sample_rates = self.sample_rates[1:]

  def __len__(self):
    return self.mel_specs.shape[0]

  def __getitem__(self, ndx):
    # returns tuple (mel spectrogram of accompaniment, mel spectrogram of vocal, rate)
    return self.mel_specs[ndx, 0], self.mel_specs[ndx, 1], self.sample_rates[ndx]

  def _mel_spectrogram(self, audio, rate):
    # compute the log-mel-spectrogram of the audio at the given sample rate
    return librosa.power_to_db(librosa.feature.melspectrogram(y = audio, sr = rate))


class LibriSpeechDataset(Dataset):

  def __init__(self, path, steps = 256, num_specs = 7647):
    self.mel_specs = self.mel_specs = torch.zeros(1, 128, steps)
    self.sample_rates = torch.tensor([0])

    num_files_opened = 0

    for speaker_dir in os.listdir(path):
      speaker_path = path + "/" + speaker_dir
      for chapter_dir in os.listdir(speaker_path):
        chapter_path = speaker_path + "/" + chapter_dir
        for file in os.listdir(chapter_path):
          # checks file extension and stops when we hit desired number of spectrograms (num_specs)
          if file.endswith('.flac') and self.mel_specs.shape[0] - 1 < num_specs:

            try:
              # get audio file and convert to log mel spectrogram
              speech, rate = librosa.load(chapter_path + "/" + file, sr = 44100)
              mel_spec = torch.from_numpy(self._mel_spectrogram(speech, rate))

              # Saves the total number of 128 x (steps) spectrograms
              num_slices = mel_spec.shape[1] // steps

              # chop off the last bit so that number of stft steps is a multiple of step size
              mel_spec = mel_spec[ : , 0 : num_slices*steps]

              # reshape the tensor to have many spectrograms of size 128 x (steps)
              mel_spec = torch.transpose(torch.reshape(mel_spec, (128, num_slices, steps)), 0, 1)

              # concatenate tensor to the full tensor in the Dataset object
              self.mel_specs = torch.cat((self.mel_specs, mel_spec), 0)
              self.sample_rates = torch.cat((self.sample_rates, torch.Tensor([rate])), 0)
              num_files_opened += 1

            except:
              print("failed to open " + file)


    # chop off the zero layer we initialized with
    self.mel_specs = self.mel_specs[1:]
    self.sample_rates = self.sample_rates[1:]
    print("opened " + str(num_files_opened) + " files")
    print("collected " + str(self.mel_specs.shape[0]) + " chunks")

  def __len__(self):
    return self.mel_specs.shape[0]

  def __getitem__(self, ndx):
    return self.mel_specs[ndx], self.sample_rates[ndx]

  def _mel_spectrogram(self, audio, rate):
    # compute the log-mel-spectrogram of the audio at the given sample rate
    return librosa.power_to_db(librosa.feature.melspectrogram(y = audio, sr = rate))

In [ ]:
path = "/content/drive/MyDrive/git_projects/spring_2025_dl_audio_project_data/"

# The string below is the path to the saved MusdbDataset in your Drive
musdbDataset_path = path + "musdb18_DatasetObject.pt"

# The string below is the path to the saved LibriSpeechDataset in your Drive
librispeechDataset_path = path + "LibriSpeechDatasetObject.pt"

musdb_dataset = torch.load(musdbDataset_path, weights_only=False)
librispeech_dataset = torch.load(librispeechDataset_path, weights_only=False)



In [ ]:
# This fixes the problem with the sample rates
musdb_dataset.sample_rates = torch.full((len(musdb_dataset),), 44100)
librispeech_dataset.sample_rates = torch.full((len(musdb_dataset),), 44100)

# Because of the way the librispeech dataset was constructed, it is slightly longer
# than the musbd dataset. Crop the librispeech dataset with these lines
librispeech_dataset.mel_specs = librispeech_dataset.mel_specs[0:len(musdb_dataset)]
librispeech_dataset.sample_rates = librispeech_dataset.sample_rates[0:len(musdb_dataset)]